#### Do standard imports for Python Data Work

In [1]:
import sys, os
from numpy import *
from matplotlib.pyplot import *
%matplotlib inline
matplotlib.rcParams['savefig.dpi'] = 100

%load_ext autoreload
%autoreload 2

#### Add working paths

In [2]:
from os.path import expanduser
import re
home = expanduser("~")
vc_dir = "Dropbox/Data Scrape of VC/"
training_dir = "NLP Training Set"

full_input_dir = os.path.join(home, vc_dir, training_dir)

## Extract all labels from training data

In [3]:
all_files = os.listdir(full_input_dir)
full_path = [os.path.join(full_input_dir, x) for x in all_files ]
files = [x for x in full_path if os.path.isfile(x)]
regex = "\[\[(.*?)\]\]"
labels = []
match_objs = []
for f in files:
    with open(f, 'r') as in_f:
        for m in re.finditer(regex, in_f.read()):
            current_tag = m.group()
            labels += [current_tag]
            match_objs += [m]
            
stripped_labels = [x[2:-2] for x in labels]
share_tags = [x.split(":")[0] for x in stripped_labels]
count_tags = [x.split(":")[1] for x in stripped_labels]
tags_to_count = {}
for label in stripped_labels:
    name, _ = label.split(":")
    count = tags_to_count.get(name, 0)
    tags_to_count[name] = count + 1

#### Here are some stats about the current labeled dataset

In [4]:
print "===== [Stats on the training data] ====="
print "Number of documents: %i" % len(files)
print "Number of training samples: %i" % len(stripped_labels)
print "Number of unique labels: %i" % len(set(share_tags))
#print "%s" % "\n ".join([str(x) for x in sorted(tags_to_count.items(), key=lambda x: x[1])])


===== [Stats on the training data] =====
Number of documents: 75
Number of training samples: 403
Number of unique labels: 36


## Stock Number Approach

Our input is a number (we assume we can extract these reliably, an assumption to be tested), an context (sliding window, sentence, and sentence parse) from which we can come up with features to find a numerican representation for the number. 

In [63]:
from tagged_file import *
tfiles = read_directory(full_input_dir)

In [64]:
# File exploration
import re
print "Total files", len(tfiles)
test_words = ['total', 'share', 'shares', 'common shares', 'common stock', 'total number', 'total stock']
# indx = {}
# for word in test_words:
#     indx[word] = []
#     for t in tfiles:
#         locs = [i for i in re.finditer(word, t.raw_clean.lower())]
#         indx[word] += [locs]
#     print "Contain", word, len([i for i in indx[word] if len(i) > 0])

# more_than_1 = [i for (i, item) in enumerate(indx['total number']) if len(item) > 1]
# print [t.context for t in tfiles[i].tags for i in more_than_1]

Total files 75


In [65]:
# RUN THIS TO MAKE OUTPUT LABELS FOR THE FIRST TASK

_round1Labels = {"TS": ['Total Shares'], "CS": ['Common Shares'], "PS": ["Preferred Shares"]}
labels = {}
for key, val in _round1Labels.iteritems():
    for v in val: 
        labels[v] = key
# labels

In [67]:
# RUN THIS TO EXTRACT ALL NUMBERS AND THEIR LABELS AND OUTPUT THEM TO all_data.json
all_data = []
doc_index = 0
for f in tfiles:
    doc_id = doc_index
    doc_index += 1
    for num in f.numbers:
        data = {}
        
        if num.label:
            data['label'] = labels.get(num.label.tag_key, 'O')
        else:
            data['label'] = 'O'
        data['before'] = num.context[0]
        data['after'] = num.context[1]
        data['doc_id'] = doc_id
        all_data += [data]
with open('all_data.json', 'w') as outp:
    json.dump(all_data, outp)
print "Processed %i numbers" % len(all_data)
# print all_data[:10]

Processed 941 numbers


In [20]:
from sklearn.feature_extraction.text import CountVectorizer


In [49]:
# Join all contexts into a big list, forming the corpus
corpus = []
for t in tfiles:
    for tag in t.tags:
        corpus += ["".join(tag.context).lower()]

In [56]:
# Construct sparse matrix representing bag of words for the corpus
sample_sentence = ['common shares preferred shares common stock preferred stock number of shares conversion']
vectorizer = CountVectorizer(min_df = 1, binary=True, ngram_range=(1,2), vocabulary=vocab)
X = vectorizer.fit_transform(corpus)



In [110]:

z = N.zeros((403, 1))
Xcpy = X.copy()

res = N.append(z, Xcpy.toarray(), axis=1)
res


array([[ 0.,  1.,  0., ...,  1.,  0.,  0.],
       [ 0.,  1.,  0., ...,  1.,  0.,  0.],
       [ 0.,  1.,  0., ...,  1.,  0.,  0.],
       ..., 
       [ 0.,  1.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.]])

In [ ]:
Xapp = N.hstack

In [74]:
import numpy as N
N.shape(Xarr)
[(x,y) for (x,y) in zip((~Xarr.any(axis=0)),
vectorizer.get_feature_names()) if x]


[(True, u'conversion'),
 (True, u'shares common'),
 (True, u'shares conversion'),
 (True, u'shares preferred'),
 (True, u'stock number')]

In [12]:
# Hypothesis: you can add the shares to find preferred and non-preferred :)

for i,t in enumerate(tfiles):
    if len(t.numbers) > 0:
        print "File index %i" % i
        print "Max number: ", max(t.numbers, key=lambda x: x.num)
        print "Tags", [tag for tag in t.tags if labels.get(tag.tag_key) == "TS"]

File index 1
Max number:  13,500,000 at 1304 (tagged)
Tags []
File index 2
Max number:  34,125,000 at 2128 (tagged)
Tags [Total Shares -> 34,125,000 @ 2128]
File index 3
Max number:  320,000,000 at 1213 (tagged)
Tags []
File index 4
Max number:  60,950,000 at 13542 (tagged)
Tags [Total Shares -> 60,950,000 @ 13542]
File index 5
Max number:  42,000,000 at 2143 (tagged)
Tags []
File index 6
Max number:  11,248,385 at 1995 (tagged)
Tags [Total Shares -> 11,248,385 @ 2003]
File index 7
Max number:  12,150,000 at 990 (tagged)
Tags [Total Shares -> 12,150,000 @ 990]
File index 8
Max number:  22,830,769 at 1406 (tagged)
Tags [Total Shares -> 22,830,769 @ 1406]
File index 9
Max number:  750099871,5 at 69809 (not tagged)
Tags [Total Shares -> 84,560,775 @ 1816]
File index 10
Max number:  63,326,579 at 1753 (tagged)
Tags [Total Shares -> 63,326,579 @ 1753]
File index 11
Max number:  14,125,849 at 1874 (tagged)
Tags [Total Shares -> 14,125,849 @ 1874]
File index 12
Max number:  50,000,000 at 6394

In [60]:
print vectorizer.get_feature_names()

[u'common', u'common shares', u'common stock', u'conversion', u'number', u'number of', u'of', u'of shares', u'preferred', u'preferred shares', u'preferred stock', u'shares', u'shares common', u'shares conversion', u'shares preferred', u'stock', u'stock number', u'stock preferred']


In [51]:
tags_by_file = [t.tags for t in tfiles]
all_tags = [item for sublist in tags_by_file for item in sublist]


In [54]:
total_tags = [tag for tag in all_tags if labels.get(tag, None) == 'TS']


[]

In [57]:
sample_sentence = all_tags[3].context
print "Parsing:", sample_sentence
result = nltk.parse.stanford(sample_sentence)

Parsing:  Preferred Stock, $0,001 par value per share (Preferred Stock), of which (i) 436,507 shares have been designated as Series A-l Convertible Preferred Stock (Series A-l Preferred Stock), (ii) 2,541,152 shares have been designated as Series A-2 Convertible Preferred Stock (Series A-2 Preferred Stock


AttributeError: 'module' object has no attribute 'stanford'

## Interesting Sentence Approach

Tokenize each document by sentences, label each sentence as interesting or not (it's interesting if it has tags) and vectorize by three features. This will be used as training data for our decision tree for whether a sentence is interesting or not. 

Currently each sentence has three features that the decision tree will use. 
- Contains a number with commas
- Sentence Length
- Contains the word 'stock' or 'share'

In [215]:
from sentence import *
sentence_objs = []
features = [ContainsCommaNumber, SentenceLength, ContainsStockOrShare]

for f in files:
    with open(f, 'r') as in_f:
        raw_text = filter_ascii(in_f.read())
        
        # Create sentence objects: see sentence.py for the way this is done
        for s in parser.tokenize(raw_text):
            sentence_objs += [Sentence(s)]


sentence_vecs = [s.to_vector(features) for s in sentence_objs]
output_labels = [s.is_interesting_as_int() for s in sentence_objs]
print "Processed %i sentences" % len(sentence_objs)
print " %i are interesting" % output_labels.count(1)


Processed 10057 sentences
 179 are interesting


In [217]:
import pylab as pl
from sklearn import tree, cross_validation
from sklearn.metrics import classification_report, confusion_matrix
clf = tree.DecisionTreeClassifier()



v_train, v_test, out_train, out_test, obj_train, obj_test = cross_validation.train_test_split(sentence_vecs, output_labels, sentence_objs, test_size=.33, random_state=0)
clf = clf.fit(v_train, out_train)
out_train_pred = clf.predict(v_train)
out_test_pred = clf.predict(v_test)

print "====== [Training report] ======"
print classification_report(out_train, out_train_pred)
cm_train = confusion_matrix(out_train, out_train_pred)
# pl.matshow(cm_train)
# pl.colorbar()
# pl.show()
print cm_train

print "====== [Test report] ====="
print classification_report(out_test, out_test_pred)
cm_test = confusion_matrix(out_test, out_test_pred)
# pl.matshow(cm_test)
# pl.colorbar()
# pl.show()
print cm_test

====== [Training report] ======
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6619
          1       0.94      0.92      0.93       119

avg / total       1.00      1.00      1.00      6738

[[6612    7]
 [   9  110]]
====== [Test report] =====
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      3259
          1       0.76      0.78      0.77        60

avg / total       0.99      0.99      0.99      3319

[[3244   15]
 [  13   47]]


####Error Analysis

In [2]:
together_train = zip(out_train_pred, out_train)
together_test = zip(out_test_pred, out_test)

false_neg_train_i = [i for i,x  in enumerate(together_train) if (x[0] == 0 and x[1] == 1)]
false_neg_test_i = [i for i, x in enumerate(together_test) if (x[0] == 0 and x[1] == 1)]

false_neg_train = [str(obj_train[i]) for i in false_neg_train_i]
false_neg_test = [str(obj_test[i]) for i in false_neg_test_i]

print "== Train =="
print "\n-------\n".join(false_neg_train)

print "--------------------------------------------------"
print "== Test =="
print "\n-------\n".join(false_neg_test)


NameError: name 'out_train_pred' is not defined

## Stage 1: Interesting Sentences

Vectorize each sentence, labelling it as interesting or not. Doing it using classification ('Interesting', 'Not') is a better long term approach because it allows for future labels to be added in addition to the stock data labelling. This might require new sentence level features to be added to `sentence.py`

# Testing Small Algorithms

#### Matching the labels within documents

In [10]:
s = "The Corporation will have the authority to issue 14,000,000[[Total Shares:14,000,000]] shares of capital stock, $0.0001 par value per share, of which 10,000,000[[Common Shares:10,000,000]] shares will be Common Stock and of which 4,000,000[[Preferred Shares:4,000,000]] shares will be Preferred Stock."

In [100]:
regex = "\[\[(.*?)\]\]"
running_length = 0
spans = []
for m in re.finditer(regex, s):
    sp = m.span()
    adjusted_pos = sp[0] - running_length
    spans += [adjusted_pos]
    running_length += sp[1] - sp[0]

print spans
without_tags = re.sub(regex, "", s)
print without_tags
print [without_tags[(x-10): x] for x in spans]


[59, 133, 184]
The Corporation will have the authority to issue 14,000,000 shares of capital stock, $0.0001 par value per share, of which 10,000,000 shares will be Common Stock and of which 4,000,000 shares will be Preferred Stock.
['14,000,000', '10,000,000', ' 4,000,000']


In [23]:
def insertChars(s, c, *indeces)
    """Insert character `c` at `indeces` in the string `s`"""
    splt = lambda s, i, : 


'[[Preferred Shares:4,000,000]]'

'abc|de|f|ghijk'

#### Tokenizing documents by sentences and matching integers

In [61]:
import nltk
parser = nltk.data.load('tokenizers/punkt/english.pickle')

In [115]:
sample_paragraph = """
RTICLE IV
Effective upon filing of this Amended and Restated Certificate of Incorporation with the Delaware Secretary of State, and prior to the issuance of any shares of Preferred Stock each currently outstanding share of Common Stock will be converted and reconstituted into 8.78204 shares of Common Stock of the Corporation (the “Stock Split”). No fractional shares or scrip representing fractional shares will be issued in connection with such Stock Split. If, after the aforementioned aggregation, the Stock Split would result in the issuance of any fractional share, the Corporation shall, in lieu of issuing any fractional share, pay cash equal to the product of such fraction multiplied by the Common Stock’s fair market value (as determined by the Board of Directors) on the date of conversion. All numbers of shares, and all amounts stated on a per share basis, contained in this Amended and Restated Certificate of Incorporation, are stated after giving effect to such Stock Split and no further adjustment shall be made as a consequence of such Stock Split.
The Corporation will have the authority to issue 14,000,000[[Total Shares:14,000,000]] shares of capital stock, $0.0001 par value per share, of which 10,000,000[[Common Shares:10,000,000]] shares will be Common Stock and of which 4,000,000[[Preferred Shares:4,000,000]] shares will be Preferred Stock.
11 7^1901? v5
"""

In [63]:
import string
filter_ascii = lambda s: filter(lambda x: x in string.printable, s)
clean_par = filter_ascii(sample_paragraph)

In [64]:
sentences = parser.tokenize(sample_paragraph)

*It's notable that the parser works pretty well. For instance, it does not split on the "." in "$0.0001 per share"*

In [68]:
print "\n ------ \n".join(sentences)


RTICLE IV
Effective upon filing of this Amended and Restated Certificate of Incorporation with the Delaware Secretary of State, and prior to the issuance of any shares of Preferred Stock each currently outstanding share of Common Stock will be converted and reconstituted into 8.78204 shares of Common Stock of the Corporation (the “Stock Split”).
 ------ 
No fractional shares or scrip representing fractional shares will be issued in connection with such Stock Split.
 ------ 
If, after the aforementioned aggregation, the Stock Split would result in the issuance of any fractional share, the Corporation shall, in lieu of issuing any fractional share, pay cash equal to the product of such fraction multiplied by the Common Stock’s fair market value (as determined by the Board of Directors) on the date of conversion.
 ------ 
All numbers of shares, and all amounts stated on a per share basis, contained in this Amended and Restated Certificate of Incorporation, are stated after giving effect 

#### NUMPY Scratch work

In [101]:
import numpy as np


In [114]:
flat_v = np.array([1, 2, 3])
tall_v = np.array([[1], [2], [3]])
reshape_v = tall_v.reshape(-1)
print flat_v.shape
print tall_v.shape
print reshape_v.shape[0]
print flat_v.reshape(-1).shape[0]

(3,)
(3, 1)
3
3


In [124]:
integer_regex = r'\d+(?:,\d+)+'
re.findall(integer_regex, sample_paragraph)

['14,000,000',
 '14,000,000',
 '10,000,000',
 '10,000,000',
 '4,000,000',
 '4,000,000']

In [134]:
sample_paragraph.lower()


'\nrticle iv\neffective upon filing of this amended and restated certificate of incorporation with the delaware secretary of state, and prior to the issuance of any shares of preferred stock each currently outstanding share of common stock will be converted and reconstituted into 8.78204 shares of common stock of the corporation (the \xe2\x80\x9cstock split\xe2\x80\x9d). no fractional shares or scrip representing fractional shares will be issued in connection with such stock split. if, after the aforementioned aggregation, the stock split would result in the issuance of any fractional share, the corporation shall, in lieu of issuing any fractional share, pay cash equal to the product of such fraction multiplied by the common stock\xe2\x80\x99s fair market value (as determined by the board of directors) on the date of conversion. all numbers of shares, and all amounts stated on a per share basis, contained in this amended and restated certificate of incorporation, are stated after givin

In [167]:
from sentence import * 
features = [ContainsCommaNumber, SentenceLength, ContainsStockOrShare]
sentences = [filter_ascii(s) for s in sentences]
sent_objs = [Sentence(s) for s in sentences]
#print sent_objs
#print [s.is_interesting() for s in sent_objs]
print [s.to_vector(features) for s in sent_objs]


[array([ 0, 53,  1]), array([ 0, 17,  1]), array([ 0, 56,  1]), array([ 0, 45,  1]), array([ 3, 35,  1]), array([0, 2, 0]), array([0, 1, 0])]
